In [1]:
import numpy as np
from losses import *

2024-12-05 16:46:58.826088: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-05 16:46:58.834306: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733438818.844034 1475844 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733438818.847043 1475844 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 16:46:58.856919: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [6]:
BATCH = 3
SEQ_LEN = 1024
N_CLASSES = 4

# random probabilities
x = np.random.rand(BATCH, SEQ_LEN, N_CLASSES)

# sparsely encoded (random) labels
y = np.random.randint(-1, N_CLASSES, (BATCH, SEQ_LEN))

In [7]:
l = MaskedTemporalCategoricalCrossEntropy(from_logits=False, sparse=True)

In [8]:
l.compute_class_weight(y[y >= 0], num_classes=N_CLASSES)

class counts: tf.Tensor([617 672 593 610], shape=(4,), dtype=int32)


<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1.0097245, 0.9270833, 1.0505903, 1.0213115], dtype=float32)>

In [9]:
np.bincount(y.flatten()+1, minlength=N_CLASSES)

array([580, 617, 672, 593, 610])